![image-alt-text](https://docs.greatexpectations.io/img/gx-logo.svg)
![image-alt-text](https://docs.greatexpectations.io/assets/images/gx_oss_process-050a4264f415a1bff3ceea3ac6f9b3a0.png)

In [ ]:
# install libraries
%pip install semantic-link great-expectations great_expectations_experimental great_expectations_zipcode_expectations

# load %%dax cell magic
%load_ext sempy

In [ ]:
import great_expectations as gx
from great_expectations.expectations.expectation import ExpectationConfiguration
from great_expectations_zipcode_expectations.expectations import expect_column_values_to_be_valid_zip5

In [ ]:
import sempy.fabric as fabric
dataset_list = fabric.list_datasets()

dataset_list

# Set up GX Data Context and Data Source

In [ ]:
# create a GX context to have access to all configurations and methods of GX
context = gx.get_context()

In [ ]:
ds = context.sources.add_fabric_powerbi("Retail Analysis Data Source", dataset="Retail Analysis")

# Specify Data Assets
**Define Data Assets to specify the subset of data you'd like to work with**
- Power BI table
- Power BI measure
- Custom DAX query
- Dynamic Management View (DMV) query

In [ ]:
# pbi table asset
ds.add_powerbi_table_asset("Store Asset", table="Store")

In [ ]:
# pbi measure asset
ds.add_powerbi_measure_asset(
    "Total Units Asset",
    measure="TotalUnits",
    groupby_columns=["Time[FiscalYear]", "Time[FiscalMonth]"]
)

In [ ]:
# custom dax query asset
ds.add_powerbi_dax_asset(
    "Total Units YoY Asset",
    dax_string=
    """
    EVALUATE SUMMARIZECOLUMNS(
        'Time'[FiscalYear],
        'Time'[FiscalMonth],
        "Total Units Ratio", DIVIDE([Total Units This Year], [Total Units Last Year])
    )    
    """
)

In [ ]:
# Dynamic Management View asset
ds.add_powerbi_dax_asset(
    "Referential Integrity Violation",
    dax_string=
    """
    SELECT
        [Database_name],
        [Dimension_Name],
        [RIVIOLATION_COUNT] 
    FROM $SYSTEM.DISCOVER_STORAGE_TABLES
    """
)

# RIVIOLATION - When key-values in a FACT table are missing from the DIMENSION table.

# Create Expectations
**Add constraints to your assets**

In [ ]:
# adding a "Retail Store Suite" with two expectations:

# a valid zip code
# a table with row count between 80 and 200
suite_store = context.add_expectation_suite("Retail Store Suite")

suite_store.add_expectation(ExpectationConfiguration("expect_column_values_to_be_valid_zip5", { "column": "PostalCode" }))
suite_store.add_expectation(ExpectationConfiguration("expect_table_row_count_to_be_between", { "min_value": 80, "max_value": 200 }))

# add expectation to context
context.add_or_update_expectation_suite(expectation_suite=suite_store)

In [ ]:
# Add a "Retail Measure Suite" with one expectation:

# Column values should be greater than 50000

suite_measure = context.add_expectation_suite("Retail Measure Suite")
suite_measure.add_expectation(ExpectationConfiguration(
    "expect_column_values_to_be_between", 
    {
        "column": "TotalUnits",
        "min_value": 50000
    }
))

# add expectation to context
context.add_or_update_expectation_suite(expectation_suite=suite_measure)

In [ ]:
# Add a "Retail DAX Suite" with one expectation:

# Column values for Total Units Ratio shoud be between 0.8 and 1.5

suite_dax = context.add_expectation_suite("Retail DAX Suite")
suite_dax.add_expectation(ExpectationConfiguration(
    "expect_column_values_to_be_between", 
    {
        "column": "[Total Units Ratio]",
        "min_value": 0.8,
        "max_value": 1.5
    }
))

# add expectation to context
context.add_or_update_expectation_suite(expectation_suite=suite_dax)

In [ ]:
# Add a "Retail DMV Suite" with one expectation:

# the RIVIOLATION_COUNT should be 0

suite_dmv = context.add_expectation_suite("Retail DMV Suite")
# There should be no RI violations
suite_dmv.add_expectation(ExpectationConfiguration(
    "expect_column_values_to_be_in_set", 
    {
        "column": "RIVIOLATION_COUNT",
        "value_set": [0]
    }
))

# add expectation to context
context.add_or_update_expectation_suite(expectation_suite=suite_dmv)

# Validation

In [ ]:
checkpoint_config = {
    "name": f"Retail Analysis Checkpoint",
    "validations": [
        {
            "expectation_suite_name": "Retail Store Suite",
            "batch_request": {
                "datasource_name": "Retail Analysis Data Source",
                "data_asset_name": "Store Asset",
            },
        },
        {
            "expectation_suite_name": "Retail Measure Suite",
            "batch_request": {
                "datasource_name": "Retail Analysis Data Source",
                "data_asset_name": "Total Units Asset",
            },
        },
        {
            "expectation_suite_name": "Retail DAX Suite",
            "batch_request": {
                "datasource_name": "Retail Analysis Data Source",
                "data_asset_name": "Total Units YoY Asset",
            },
        },
        {
            "expectation_suite_name": "Retail DMV Suite",
            "batch_request": {
                "datasource_name": "Retail Analysis Data Source",
                "data_asset_name": "Referential Integrity Violation",
            },
        },
    ],
}
checkpoint = context.add_checkpoint(
    **checkpoint_config
)


In [ ]:
result = checkpoint.run()

In [ ]:
import pandas as pd

data = []

for run_result in result.run_results:
    for validation_result in result.run_results[run_result]["validation_result"]["results"]:
        row = {
            "Batch ID": run_result.batch_identifier,
            "type": validation_result.expectation_config.expectation_type,
            "success": validation_result.success
        }

        row.update(dict(validation_result.result))
        
        data.append(row)

result_df = pd.DataFrame.from_records(data)    

result_df[["Batch ID", "type", "success", "element_count", "unexpected_count", "partial_unexpected_list"]]

# Diagnostics
**Understand which exact years are out of range**

In [ ]:
# Use semantic link to execute the same query you passed into the GX Data Asset and visualize the resulting values.

dataset = "Retail Analysis"

dax = """
    EVALUATE SUMMARIZECOLUMNS(
        'Time'[FiscalYear],
        'Time'[FiscalMonth],
        "Total Units Ratio", DIVIDE([Total Units This Year], [Total Units Last Year])
    )    
    """

df = fabric.evaluate_dax(dataset,dax)
df

In [ ]:
import matplotlib.pyplot as plt

df["Total Units % Change YoY"] = (df["[Total Units Ratio]"])

df.set_index(["Time[FiscalYear]", "Time[FiscalMonth]"]).plot.bar(y="Total Units % Change YoY")

plt.axhline(0)

plt.axhline(0.8, color="red", linestyle="dotted")
plt.axhline( 1.5, color="red", linestyle="dotted")

None